In [ ]:
#Script para gerar Tabela de Notas Fiscais, Itens de nota e Vendas(OLTP), buscando o IDPRDOUTO na tabela Produto do banco de dados

import random
import pandas as pd
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"


conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

# Função para gerar data e hora aleatória
def gerar_data_hora(ano_inicio, ano_fim):
    ano = random.randint(ano_inicio, ano_fim)
    mes = random.randint(1, 12)
    if mes in [4, 6, 9, 11]:  # Meses com 30 dias
        dia = random.randint(1, 30)
    elif mes == 2:  # Fevereiro
        dia = random.randint(1, 28)
    else:  # Meses com 31 dias
        dia = random.randint(1, 31)

    hora = random.randint(8, 22)
    minuto = random.randint(0, 59)
    segundo = random.randint(0, 59)

    data_completa = datetime(ano, mes, dia, hora, minuto, segundo)
    return data_completa

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    # Garantir que a quantidade de itens únicos seja pelo menos 1
    quant_item_unicos = random.randint(1, quant_maxima_itens)  # Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")

    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    lista_NOTAFISCAL = []
    

    id_forma_pgmt = random.randint(1, 8)
    id_cliente = random.randint(1, 1000)
    id_loja = random.randint(1, 150)
    id_cfop = random.randint(1, 5)
    chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(15)])
    data_criacao = data_nota.date()
    data_atlz = ""
    versao = 1
    
    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        print(produto)
        if produto in df_prod["NOME"].values:
            preco_unit = df_prod.loc[df_prod["NOME"] == produto, "PRECO_UNIT"].values[0]
            id_produto = df_prod.loc[df_prod["NOME"] == produto, "IDPRODUTO"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((data_nota, numero_nota, produto, quantidade, preco_unit, preco_total_item, 
                                id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, data_atlz, versao,id_produto))
            

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)
    
    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]
   
    lista_NOTAFISCAL.append((data_nota, id_forma_pgmt, id_cliente, id_loja, id_cfop, chaveacesso, data_criacao, 
                             data_atlz, versao,preco_total_nota))
    

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida, columns=["DATA_NOTAFISCAL", "Numero_Nota", "Produto", "QUANTIDADE", "PRECO_UNIT", "TOTAL_ITEM", 
                              "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP","CHAVE_ACESSO","DATA_CRIACAO",
                              "DATA_ATLZ","VERSAO", "IDPRODUTO","TOTAL_NOTA"]
    )

    df_2 = pd.DataFrame(
        lista_NOTAFISCAL, columns=["DATA_NOTAFISCAL", "ID_FORMA_PGMT", "ID_CLIENTE", "ID_LOJA", "ID_CFOP", 
                                   "CHAVE_ACESSO", "DATA_CRIACAO", "DATA_ATLZ", "VERSAO","TOTAL_NOTA"]
    )

    
    return df_nota_fiscal, df_2

# Lista de produtos e lista de ID PRODUTO a partir do banco de dados

df_prod = pd.read_sql(text(f"SELECT IDPRODUTO, NOME, PRECO_UNIT FROM PRODUTO "),engine)
display(df_prod)

lista_IDPRODUTO = df_prod['IDPRODUTO'].to_list()
lista_prod_DB = df_prod['NOME'].to_list()

#print(lista_IDPRODUTO)
#print(len(lista_IDPRODUTO))

## Exemplo de DataFrame de produtos e preços unitários
#data = {
#    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
#    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
#}
#df_prod_valores = pd.DataFrame(data)

#rewmover
#listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"]

# Quantidade de notas fiscais
numero_inicial_nota = 1000
numero_maximo_nota = 1003
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []
df_notas_fiscais_2 = []


# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = gerar_data_hora(2024, 2024)
    df_nota_fiscal, df_2 = gerar_nota_fiscal(numero_nota, lista_prod_DB, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)
    df_notas_fiscais_2.append(df_2)
   


# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)
df_todas_notas2 = pd.concat(df_notas_fiscais_2, ignore_index=True)


df_ITEMNOTA = pd.DataFrame({
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_PRODUTO':     df_todas_notas['IDPRODUTO'],
                'QUANTIDADE':     df_todas_notas['QUANTIDADE'],
                'PRECO_UNIT':     df_todas_notas['PRECO_UNIT'],
                'TOTAL_ITEM':     df_todas_notas['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

df_NOTAFISCAL = pd.DataFrame({
                'DATA':           df_todas_notas2['DATA_NOTAFISCAL'],
                'TOTAL_NOTA':     df_todas_notas2['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_todas_notas2['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_todas_notas2['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas2['ID_LOJA'],
                'ID_CFOP':        df_todas_notas2['ID_CFOP'],
                'CHAVE_ACESSO':   df_todas_notas2['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_todas_notas2['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas2['DATA_ATLZ'],
                'VERSAO':         df_todas_notas2['VERSAO'],
})

df_VENDA = pd.DataFrame({
                'DATA':           df_todas_notas['DATA_NOTAFISCAL'],
                'ID_NOTAFISCAL':  df_todas_notas['Numero_Nota'],
                'ID_CLIENTE':     df_todas_notas['ID_CLIENTE'],
                'ID_LOJA':        df_todas_notas['ID_LOJA'],
                'ID_FORMA_PGMT':  df_todas_notas['ID_FORMA_PGMT'],
                'TOTAL_VENDA':     df_todas_notas['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_todas_notas['DATA_CRIACAO'],
                'DATA_ATLZ':      df_todas_notas['DATA_ATLZ'],
                'VERSAO':         df_todas_notas['VERSAO'],
})

#Exibir todas as notas fiscais
#print("\nTodas as Notas Fiscais:")
#print(df_todas_notas)

display(df_ITEMNOTA)
#display(df_NOTAFISCAL)
#display(df_VENDA)
#

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida


,IDPRODUTO,NOME,PRECO_UNIT
0,1,Limpa-azulejo Seda,8.77
1,2,Mexilhao Chocolandia,30.11
2,3,Removedor de manchas Santo Antonio,7.00
3,4,Queijo parmesao Seara,63.21
4,5,Rodo de pia Tio Joao,16.62
...,...,...,...
85979,85980,Salsicha hot dog Nivea,14.22
85980,85981,Oleo de gergelim Indaia,25.99
85981,85982,Ovos Rexona,9.75
85982,85983,Morango Garoto,12.79


Quantidade de itens únicos na nota 1000: 6
Porta-sabao Seara
Desinfetante para banheiro Hellmann's
Farinha de amendoas Nugget
Limpa-vidros Panco
Locao pos-barba Chester
Esponja de lavar louca Marta Rocha
Quantidade de itens únicos na nota 1001: 2
Queijo coalho Tigrao
Abacaxi Seda
Quantidade de itens únicos na nota 1002: 3
Panela de pressao Carmen
Figo Seara
Protetor labial Borges
Quantidade de itens únicos na nota 1003: 9
Biscoito recheado Itaipava
Lula Coqueiro
Vinho Passoca
Desinfetante para banheiro Delmiro
Frigideira Nugget
Melancia Rei do Mate
Pipoca colorida Colgate
Limpador para pisos Kraft Heinz
Salsicha vegetariana Coca-Cola


,ID_NOTAFISCAL,ID_PRODUTO,QUANTIDADE,PRECO_UNIT,TOTAL_ITEM,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,1000,54771,5,10.40,52.00,2024-03-21,,1
1,1000,46751,7,6.13,42.91,2024-03-21,,1
2,1000,49299,6,23.50,141.00,2024-03-21,,1
3,1000,6582,9,6.64,59.76,2024-03-21,,1
4,1000,56073,5,45.45,227.25,2024-03-21,,1
5,1000,40851,2,2.74,5.48,2024-03-21,,1
6,1001,29225,2,15.94,31.88,2024-07-24,,1
7,1001,66375,1,6.89,6.89,2024-07-24,,1
8,1002,73199,4,265.55,1062.20,2024-10-08,,1
9,1002,12615,6,22.28,133.68,2024-10-08,,1


In [ ]:
#Script para gerar Tabela Item Nota

import random
import pandas as pd

# Exemplo de DataFrame de produtos e preços unitários
data = {
    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
}
df_prod_valores = pd.DataFrame(data)

# Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens):
    # Gerar quantidade aleatória de itens únicos na nota fiscal
    quant_item_unicos = random.randint(1, quant_maxima_itens)
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")
    
    lista_saida = [] # Lista para armazenar os itens da nota fiscal

    # Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod_valores["NOME"].values:
            preco_unit = df_prod_valores.loc[df_prod_valores["NOME"] == produto, "PRECO_UNIT"].values[0]
            preco_unit = round(float(preco_unit), 2)  # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)  # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            lista_saida.append((numero_nota, produto, quantidade, preco_unit, preco_total_item))

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[4] for item in lista_saida), 2)

    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]

    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida,
        columns=["Número da Nota", "Produto", "Quantidade", "Preço Unitário", "Total do Item", "Total da Nota"]
    )

    return df_nota_fiscal

# Lista de produtos
listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle", "Pepsi", "Guarana Antarctica"]

# Gerar uma nota fiscal
numero_nota = 1000  # Número da nota fiscal
quant_maxima_itens = 10
df_nota_fiscal = gerar_nota_fiscal(numero_nota, listaprodutosPROVISORIO, quant_maxima_itens)

# Exibir a nota fiscal
print("\nNota Fiscal:")
print(df_nota_fiscal)
print(f"\nTotal da Nota: R$ {df_nota_fiscal['Total da Nota'].iloc[0]:.2f}")

In [ ]:
#Script para gerar Tabela de Notas Fiscais

import random
import pandas as pd
from datetime import datetime

# Exemplo de DataFrame de produtos e preços unitários
data = {
    "NOME": ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle"],
    "PRECO_UNIT": [10.0, 15.5, 8.0, 12.3, 9.8]
    
}
df_prod_valores = pd.DataFrame(data)

def gerar_data_hora(ano_inicio, ano_fim):
    ano = random.randint(ano_inicio, ano_fim)
    mes = random.randint(1, 12)
    if mes in [4, 6, 9, 11]:  
        dia = random.randint(1, 30)
    elif mes == 2:  
        dia = random.randint(1, 28)
    else:  
        dia = random.randint(1, 31)

    hora = random.randint(8, 22)
    minuto = random.randint(0, 59)
    segundo = random.randint(0, 59)

    data_completa = datetime(ano, mes, dia, hora, minuto, segundo)
    
    return data_completa

#Função para gerar a nota fiscal
def gerar_nota_fiscal(numero_nota, lista_produtos, quant_maxima_itens, data_nota):
    
    quant_item_unicos = random.randint(1, quant_maxima_itens) #Gerar quantidade aleatória de itens únicos na nota fiscal
    print(f"Quantidade de itens únicos na nota {numero_nota}: {quant_item_unicos}")
    
    lista_saida = []  # Lista para armazenar os itens da nota fiscal
    
    id_forma_pgmt = random.randint(1,8)
    id_cliente = random.randint(1,1000)
    id_loja = random.randint(1,150)
    id_cfop = random.randint(1,5)

    #Gerar os itens da nota fiscal
    for _ in range(quant_item_unicos):
        produto = random.choice(lista_produtos)
        if produto in df_prod_valores["NOME"].values:
            preco_unit = df_prod_valores.loc[df_prod_valores["NOME"] == produto, "PRECO_UNIT"].values[0]
            preco_unit = round(float(preco_unit), 2)              # Preço unitário com 2 casas decimais
            quantidade = random.randint(1, 10)                    # Quantidade aleatória do produto
            preco_total_item = round(quantidade * preco_unit, 2)  # Total do item
            data = data_nota
            id_forma_pgmt = id_forma_pgmt
            id_cliente = id_cliente
            id_loja = id_loja
            id_cfop = id_cfop
            lista_saida.append((data, numero_nota, produto, quantidade, preco_unit, preco_total_item, id_forma_pgmt, id_cliente, id_loja, id_cfop))

    # Calcular o total da nota fiscal
    preco_total_nota = round(sum(item[5] for item in lista_saida), 2)

    # Adicionar o total da nota a cada item da lista
    lista_saida = [item + (preco_total_nota,) for item in lista_saida]

    #Data da nota fiscal
    #data_nota = gerar_data_hora(2024,2024)


    # Criar um DataFrame com os dados da nota fiscal
    df_nota_fiscal = pd.DataFrame(
        lista_saida,
        columns=["Data","Número da Nota", "Produto", "Quantidade", "Preço Unitário", "Total do Item","Id Forma Pgmt","Id Cliente","Id Loja","Id CFOP", "Total da Nota"]
    )

    return df_nota_fiscal

# Lista de produtos
listaprodutosPROVISORIO = ["Coca-Cola", "Sadia", "Codil", "Tres Coracoes", "Nestle", "Pepsi", "Guarana Antarctica"]

#Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 1005  
quant_maxima_itens = 10

# Lista para armazenar todas as notas fiscais
df_notas_fiscais = []

# Gerar notas fiscais para o intervalo definido
for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
    data_nota = gerar_data_hora(2024,2024)
    df_nota_fiscal = gerar_nota_fiscal(numero_nota, listaprodutosPROVISORIO, quant_maxima_itens, data_nota)
    df_notas_fiscais.append(df_nota_fiscal)

# Concatenar todas as notas fiscais em um único DataFrame
df_todas_notas = pd.concat(df_notas_fiscais, ignore_index=True)

# Exibir todas as notas fiscais
print("\nTodas as Notas Fiscais:")
print(df_todas_notas)

# Exibir o total de cada nota fiscal
#print("\nResumo dos Totais das Notas Fiscais:")
#for numero_nota in range(numero_inicial_nota, numero_maximo_nota + 1):
#    total_nota = df_todas_notas.loc[df_todas_notas["Número da Nota"] == numero_nota, "Total da Nota"].iloc[0]
#    print(f"Nota {numero_nota}: R$ {total_nota:.2f}")